# Burry

Burry is a project to analyse text data, in the form of tweets, to find relation between those texts and subjects.

It is done as a project for the Fontys University of ICT and the Central Bureau of Statistics of the Netherlands.

Currently it is focussed at detecting sickness.

In [1]:
import pandas as pd
import numpy as np
import operator
import re
from collections import Counter
import gensim
from gensim import corpora, models
from gensim.models.word2vec import Word2Vec

doc = pd.read_csv('sample_dataset.csv', sep=';')

search_terms = [re.sub(r'(src:\w+)', '', word).strip() for word in doc['zoekopdracht'].unique()]
print(search_terms)

doc.head()

['ziek']


zoekopdracht             datum  \
0  ziek src:twitter  2017-09-20 10:11   
1  ziek src:twitter  2017-09-20 10:11   
2  ziek src:twitter  2017-09-20 10:10   
3  ziek src:twitter  2017-09-20 10:09   
4  ziek src:twitter  2017-09-20 10:07   

                                                 url sentiment     type  \
0  https://twitter.com/HubertDeMeulder/status/910...         -     post   
1  https://twitter.com/michielvdbroeck/status/910...         +  comment   
2  https://twitter.com/petervalk1/status/91041589...         -  comment   
3  https://twitter.com/Korneel_Evers/status/91041...         +  comment   
4  https://twitter.com/sariehimpe/status/91041518...         -     post   

   discussielengte    views           auteur  volgers  invloed  \
0              NaN  14671.0  HubertDeMeulder    14671      4.8   
1              4.0      NaN  michielvdbroeck     1492      1.7   
2              2.0   1088.0       petervalk1     1088      1.8   
3            644.0      NaN    Korneel_Evers     4411     21.4   
4              NaN    332.0       sariehimpe      332      0.4   

   GPS breedtegraad  GPS lengtegraad  \
0               NaN              NaN   
1               NaN              NaN   
2               NaN              NaN   
3               NaN              NaN   
4               NaN              NaN   

                                       bericht tekst type bron  titel  
0  #SabineHagedoren: “Op het werk wist niemand da...   twitter    NaN  
1                    @heloisesell Haha wtf ziek nice   twitter    NaN  
2  RT @AcrisiuS322: Van der Laan is ziek gemeld, ...   twitter    NaN  
3  @_jazzybelle @zoekpostadres @real_Raffie @olaf...   twitter    NaN  
4  Iedereen gaat al een week de tijd gehad hebben...   twitter    NaN

# Cleaning data
Here we do several things:
* remove retweets
* convert text to lowercase

In the context of our research a retweet does not add any relevant data. It actually skews the results since a specific phrasing get's repeated more often than it naturally would.

In [2]:
doc['bericht tekst'] = doc['bericht tekst'].fillna('')

In [3]:
doc['bericht tekst'] = doc['bericht tekst'].str.lower()

In [4]:
doc  = doc[~doc['bericht tekst'].str.contains('rt')]
doc  = doc[~doc['bericht tekst'].str.contains(r'http[s]*')]  # TODO secure?
doc  = doc[~doc['auteur'].str.contains('grieptweets')]  # TODO to file
doc  = doc[~doc['auteur'].str.contains('kleenex_helpt')]
# doc.head()

In [5]:
doc = doc.drop_duplicates()
len(doc)

6530

# Defenition of helper function


In [6]:
re_clean = re.compile(r'(https?://\S+|@\S+)')
re_words = re.compile(r'(\w+-?\w*)')

def clean_text(text: str):
    words = []
    if text:
        text = re_clean.sub(' ', text)
        words = re_words.findall(text)
    return words

In [7]:
all_tweets = pd.read_csv('sample_alltweets.csv', sep=';')
all_tweets.head()

zoekopdracht             datum  \
0  src:twitter  2017-09-19 13:09   
1  src:twitter  2017-09-20 10:47   
2  src:twitter  2017-09-29 13:07   
3  src:twitter  2017-10-02 21:09   
4  src:twitter  2017-09-12 07:13   

                                                 url sentiment     type  \
0  https://twitter.com/Savage_Grethan/status/9100...       NaN  comment   
1  https://twitter.com/PolBegov/status/9104252503...       NaN     post   
2  https://twitter.com/robbertnootnoot/status/913...       NaN     post   
3  https://twitter.com/BanessaFabulous/status/914...       NaN  comment   
4  https://twitter.com/Radio_20/status/9074721078...       NaN     post   

   discussielengte   views           auteur  volgers  invloed  \
0              2.0   844.0   Savage_Grethan      844      0.4   
1              NaN  2779.0         PolBegov     2779      0.9   
2              NaN   300.0  robbertnootnoot      300      6.3   
3            456.0     NaN  BanessaFabulous      102      0.6   
4              NaN  2055.0         Radio_20     2055      1.5   

   GPS breedtegraad  GPS lengtegraad  \
0               NaN              NaN   
1               NaN              NaN   
2               NaN              NaN   
3               NaN              NaN   
4               NaN              NaN   

                                       bericht tekst type bron  titel  
0   RT @thedolanclones: SJKSK IM QUAKIHG IN MY BOOTS   twitter    NaN  
1  VUB hanteert quotum voor vrouwelijke professor...   twitter    NaN  
2  Thank the Gods we're going to wave goodbye to ...   twitter    NaN  
3  @nytimes @rcallimachi It really does not matte...   twitter    NaN  
4  #Media companies must reimagine their #Data fo...   twitter    NaN

# Refining data
Split a tweet into words. Since individual words are easier to process.

In [8]:
doc['bericht woorden'] = doc['bericht tekst'].map(clean_text)

# Cleaning data
Here we do several things:
* convert messages with no content with to the message `''`
* remove retweets
* convert to lowercase

The conversion is needed since the `clean_text()` function expects a string.

In [9]:
all_tweets['bericht tekst'] = all_tweets['bericht tekst'].fillna('')

In [10]:
all_tweets['bericht tekst'] = all_tweets['bericht tekst'].str.lower()

In [11]:
all_tweets = all_tweets[~all_tweets['bericht tekst'].str.contains('rt')]
all_tweets.head()

zoekopdracht             datum  \
2   src:twitter  2017-09-29 13:07   
3   src:twitter  2017-10-02 21:09   
4   src:twitter  2017-09-12 07:13   
7   src:twitter  2017-10-04 15:31   
11  src:twitter  2017-09-26 19:18   

                                                  url sentiment     type  \
2   https://twitter.com/robbertnootnoot/status/913...       NaN     post   
3   https://twitter.com/BanessaFabulous/status/914...       NaN  comment   
4   https://twitter.com/Radio_20/status/9074721078...       NaN     post   
7   https://twitter.com/Drflsn/status/915569982007...       NaN  comment   
11  https://twitter.com/handige_henny/status/91272...       NaN     post   

    discussielengte   views           auteur  volgers  invloed  \
2               NaN   300.0  robbertnootnoot      300      6.3   
3             456.0     NaN  BanessaFabulous      102      0.6   
4               NaN  2055.0         Radio_20     2055      1.5   
7               NaN     NaN           Drflsn      758      0.3   
11              NaN   212.0    handige_henny      212      2.1   

    GPS breedtegraad  GPS lengtegraad  \
2                NaN              NaN   
3                NaN              NaN   
4                NaN              NaN   
7                NaN              NaN   
11               NaN              NaN   

                                        bericht tekst type bron  titel  
2   thank the gods we're going to wave goodbye to ...   twitter    NaN  
3   @nytimes @rcallimachi it really does not matte...   twitter    NaN  
4   #media companies must reimagine their #data fo...   twitter    NaN  
7           @westerns_sq @skatemlley wellllkammmmmmmm   twitter    NaN  
11  ik heb een video toegevoegd aan een @youtube-a...   twitter    NaN

In [12]:
all_tweets['bericht woorden'] = all_tweets['bericht tekst'].map(clean_text)

# Learning

Now we start learning from our dataset.

First we will find the most common words in our set, so we know which ones to ignore. Think like "the" and "it" in english.

We will put these word's in our blacklist anlong with our search terms.

We are using the dataset without filters to scan all possible tweets.


In [13]:
execute_learning = False

In [14]:
result = []
if execute_learning: 
    counter = Counter()
    for words in all_tweets['bericht woorden']:
        counter.update(words)
    result = counter.most_common(25)
result

[]

In [15]:
if execute_learning: 
    common_words = set([word[0] for word in counter.most_common(300)])

In [16]:
if execute_learning: 
    blacklisted_words = set(common_words)
    blacklisted_words.update(set(search_terms))

# Related words
Now we will try to find related words to our search terms.

We will be doing this using our blacklist to filter out the blacklisted and common words.

Essentially this also is a common_words list but then to find the relations instead of common words.

These words are not used by the actual model. However they serve as a help for defining relevant search words.

In [17]:
related_words = []
if execute_learning: 
    counter = Counter()

    for words in doc['bericht woorden']:
        words = set(words)
        filtered_words = words - blacklisted_words
        counter.update(filtered_words)

    related_words = counter.most_common(25)
related_words

[]

# Training
Now we will train the model using a dataset from wikipedia.

This dataset is used to replicate normal language and analyse the relations between words.

We have already trained a model and will be loading it in here

In [18]:
model = Word2Vec.load('word2vec.model')

# Analysing
Now we will analyse our dataset using our model.

We will check for similarity of the tweets against our sickness_terms to score a tweet to being from a sick person.


In [ ]:
sickness_terms = [
    'ziek',
    'griep',
    'verkouden',
    'verkoudheid',
    'koorts',
    'hoofdpijn',
]

In [20]:
def scorer(row):
    if 'score' not in row:
        score = 0
        words = [word.replace('#', '') for word in row['bericht woorden'] if word in model.wv.vocab]
        if words:
            score = model.wv.n_similarity(sickness_terms, words)
        if row['type'] == 'comment':
            score /= 2
        row['score'] = score
    return row

doc = doc.apply(scorer, axis=1)
doc = doc.sort_values('score', ascending=False)

In [21]:
pd.set_option('display.max_colwidth', 250)
pd.options.display.max_rows = 999

# Visualization
As you can see below, the model is pretty good at finding the relevant tweets.

The problem we face now is that Dutch people like the swear with deceases.
 - Ziek volkje -> sick people (in the sense of not thinking straight)
 - Tering ziek -> Tuberculosis sick (ambiguous: could be really sick, or I can't understand why)

In [22]:
doc.filter(items=['bericht tekst',  'score', 'auteur', 'type']).head(250)

bericht tekst  \
3213                                                                                                                                     tyfus ziek   
4668                                                                                                             ziek erge hoofdpijn.. iemand tips?   
9679   voor 2e dag ziek thuis. maag/darmen teveel prikkels. heerst er weer iets? #buikgriep #griep #ziek #ziekthuis #zzm #ziekzwakmisselijk #gatver   
2185                                                                                                                ziek thuis griep en verkouden 😷   
13391                                                                                            hoofdpijn, buikpijn, moe tekenen van ziek komen??😞   
12865                                                                                                                              ziek veel stress   
4767                                                                                           hij is misschien psychisch ziek "ingebeelde ziekten"   
17373                                                                                                                                  ziek volkje.   
17507                                                                                                                                 tering ziek..   
17929                                                                                                                         smerig ziek klotevolk   
3435                                                                                                                                     bitje ziek   
3466                                                                                                                            ziek gestoord tuig.   
17962                                                                                                                          ziek thuis met griep   
16079                                                                                                          #ziek  doorgedraaid lijstjessysteem.   
8657                                                                                   kapot misselijk, ziek erge hoofdpijn & kan niet slapen. nice   
4100                                                                                                                                    fcking ziek   
13140                                                                                                                                    ziek... :(   
13918                                                                                                                                        ziek 😪   
18490                                                                                                                                        ziek 😷   
17577                                                                                                                                      ziek...!   
8674                                                                                                                                           ziek   
13121                                                                                                                                          ziek   
7050                                                                                                                                           ziek   
13197                                                                                                                                         ziek!   
13159                                                                                                                                        ziek 😪   
16099                                                                                                                                          ziek   
1457                                                     